# Environmental items

If you receive errors that a module cannot be found you can install the modules we'll be using from with the command line with the following:

`mamba install -c conda-forge mysql-connector-python`

`conda install -c anaconda sqlalchemy OR pip install sqlalchemy`

# name == "main":

Python files are referred to as “modules”. A module can have functions, classes, and variables. When the Python interpreter runs a module, the `__name__` variable will be set as `__main__` if the module being run is the main program. But if the code is being run from another module, then the `__name__` variable will be set to that module’s (file) name.

In Python, the if **name** == '**main**': block is a conditional statement that checks whether the current module (Python file) is being run as the main program or if it is being imported as a module into another program.

Here's how it works:

1. Every Python module has a special variable called **name**. When a module is executed as the main program, the **name** variable is set to the string '**main**'. However, if the module is being imported into another program, the **name** variable is set to the name of the module.
2. The if **name** == '**main**': block allows you to specify code that should only be executed when the module is run as the main program. This block is typically placed at the end of the module, after all the function and class definitions.
3. Any code inside the if **name** == '**main**': block will be executed only if the module is run directly as the main program. If the module is imported into another program, the code inside this block will be skipped.

Specifying this variable allows us to control which module the Python interpreter should consider the “main” module when there are multiple modules that are imported into other modules.

### We'll create two files: `module1.py` and `module2.py`

---

# Database setup

Before we can begin writing our program to interact with the database, we must ensure the database exists and has tables with the appropriate structure. The database to use will be ‘infs3070’ and each student can create their own database table by either using PHPMyAdmin, or by using the SQL script from the command line.

## Create the database

In [ ]:
CREATE DATABASE infs3070;

## Create the database table

In [ ]:
CREATE TABLE `infs3070`.`students` (
    `StudentID` INT NOT NULL AUTO_INCREMENT , 
    `lastName` VARCHAR(45) NULL , 
    `firstName` VARCHAR(45) NULL , 
    `email` VARCHAR(45) NULL , 
    PRIMARY KEY (`StudentID`));

## Create a user allowed to access database

In [ ]:
-- create the user (uses default plugin: caching_sha2_password)
CREATE USER 'infs3070'@'%' IDENTIFIED BY 'pydev';

-- give privileges ONLY on the INFS3070 schema
GRANT ALL PRIVILEGES ON `INFS3070`.* TO 'infs3070'@'%';

FLUSH PRIVILEGES;

# Connecting to database using Python

We are going to cover two options for connecting:

1. Provide connection details directly within our Python file that needs access.
2. Call a function created in a separate file that connections connection parameters. (*My preferred*)

When we want to interact with a database using Python, it is common for us to use a connection object (`conn = create_conn()`) to create the connection to the database, and a cursor object (`cursor = conn.cursor()`) to interact with the database once connected.

This connection creates "handles" to finite system resources (network sockets, server processes, memory, etc.) Therefore, when we have finished using these connections we should close the connections to free up the system resources used by them. `cursor.close()` and `conn.close()`.

A 'cursor' is a lightweight object that performs the following important tasks:
- Sends SQL statements to the database through our connection.
- Keeps track of the current position in the results.
- Can hold memory buffers both client and server side to prepare for results.

## Option 1

In [ ]:
import mysql.connector
# import os

def create_conn():
    conn = mysql.connector.connect(
        host="localhost",
        user="infs3070",
        # user = os.environ.get('3070User'),
        password="pydev",
        # password = os.environ.get('3070Pass'),
        database="infs3070"
    )
    return conn


# There would be code below this line....


## Option 2

### Create a separate connection file: `dbConfig.py`

In [ ]:
import mysql.connector

def create_conn():
    conn = mysql.connector.connect(
        host="localhost",
        user='infs3070',
        password='pydev',
        database='infs3070'
    )
    return conn



## Using dbConfig to connect

### Select data from our database.

In [ ]:
from dbConfig import create_conn

# Create a function to select all the records from our database
def get_all_students():
    conn = create_conn()
    cursor = conn.cursor()

    select_query = 'select * from students;'
    cursor.execute(select_query)

    results = cursor.fetchall()
    print("All student records: ")
    for row in results:
        print(row)

    cursor.close()
    conn.close()

if __name__ == "__main__":
    get_all_students()


All student records: 
(1, 'Smith', 'John', 'js@email.com')
(2, 'jingleheimer', 'john', 'jj@emai.com')
(3, 'poppins', 'mary', 'mp@email.com')
(4, 'theclown', 'blinky', 'bc@email.com')


### Inserting records into the database.

In [ ]:
from dbConfig import create_conn

# Function for inserting records into the database
def add_student(last_name, first_name, email):
    conn = create_conn()
    cursor = conn.cursor()

    # Store the values as placeholders for the function
    insert_query = 'insert into students (lastName, firstName, email) values (%s, %s, %s)'
    record = (last_name, first_name, email)

    # Use the information to populate the database
    cursor.execute(insert_query, record)
    conn.commit()

    print("Student record added successfully!")
    
    cursor.close()
    conn.close()


# Provide data to insert
if __name__ == "__main__":
    last_name = "theclown"
    first_name = "blinky"
    email = "bc@email.com"

    add_student(last_name, first_name, email)

Student record added successfully!


### Updating database records

In [ ]:
from dbConfig import create_conn

# Function for updating
def update_student(student_id, last_name, first_name, email):
    conn = create_conn()
    cursor = conn.cursor()

    update_query="""
        update students
        set lastName = %s, firstName = %s, email = %s
        where studentID = %s;
    """

    cursor.execute(update_query, (last_name, first_name, email, student_id))
    conn.commit()


    print("Student record updated successfully!")
    cursor.close()
    conn.close()

# Add the record information to update
student_id = 1
updated_last_name = 'Smith'
updated_first_name = 'John'
updated_email = 'js@email.com'
update_student(student_id, updated_last_name, updated_first_name, updated_email)



Student record updated successfully!


### Delete records

In [ ]:
from dbConfig import create_conn

# Create our delete function
def delete_student(student_id):
    conn = create_conn()
    cursor = conn.cursor()

    delete_query = "delete from students where studentID = %s;"
    cursor.execute(delete_query, (student_id,))

    conn.commit()

    if cursor.rowcount > 0:
        print("Student record deleted successfully!")
    else:
        print("No record found with the given student ID.")

    cursor.close()
    conn.close()


# Add information for the record to delete
student_id = 4
delete_student(student_id)

No record found with the given student ID.


# Connecting to class database server for `companyDB`

Creating a connection to a database with Pandas is very similar to how we connect with "vanilla" python. The difference is when we want to interact with the data through Pandas for analysis purposes.

To perform analysis we will not only use Pandas and the mySQL connector, but also a new library called 'sqlalchemy'.

SQLAlchemy is a popular and powerful Object Relational Mapper (ORM) and SQL toolkit library for Python. It provides a set of high-level API and tools to communicate with relational databases.

We will use the `create_engine` function within SQLAlchemy to create a new connection ("engine") to the database.

## Using sqlalchemy

In [1]:
from companyConnect import get_conn_params
import pandas as pd
from sqlalchemy import create_engine, text

params = get_conn_params()

# Construct SQLAlchemy connection URL
engine = create_engine("mysql+pymysql://{user}:{password}@{host}/{database}".format(**params))

df = pd.read_sql("select count(*) from sales;", engine)

print(df)

   count(*)
0    367213
